# Мютексы

В пакете sync стандартной библиотеки Go есть тип Mutex, который можно использовать для реализации синхронного доступа к данным. Наш пример из предыдущей главы:

In [ ]:
import "sync"

var (
	mu      sync.Mutex
	balance int
)

func Deposit(amount int) {
	mu.Lock()
	balance += amount
	mu.Unlock()
}

func Balance() int {
	mu.Lock()
	defer mu.Unlock()
	return balance
}

Мютекс mu защищает разделяюмую переменную balance вызовами Lock и Unlock, чтобы только одна горутина могла иметь доступ к этой переменной в конкретный момент времени. Обычно защищаемую переменную размещают сразу за мютексом, который используется для её защиты. Если это правило не соблюдается, то необходимо чётко это задокументировать.

Область между методами Lock и Unlock называют **критической секцией**(critical section).

В Go также есть мютекс, который позволяет вычитывать данные одновременно нескольким горутинам, но предоставляет эксклюзивный доступ при записи. Реализуется такое поведение с помощью Read/Write мютекса из пакета sync: sync.RWMutex.

Примитивы синхронизации не только приводят к синхронному доступу к данным, но также сбрасывают(flush) кэш процессора, чтобы все накопленные изменения переменных стали видны и другим горутинам. Как это ни странно, но следующий код в теории может привести к выводу вида "x:0 y:0" или "y:0 x:0":

In [ ]:
var x, y int

go func() {
    x = 1
    fmt.Print("y:", y, " ")
}()

go func() {
    y = 1
    fmt.Print("x:", x, " ")
}()